# Import Library

In [1]:
#!pip install datasets
from datasets import load_dataset

#dataset = load_dataset("csebuetnlp/xlsum",'indonesian')
import tensorflow as tf
from tensorflow.keras import callbacks, models, layers, preprocessing as kprocessing #(2.6.0)
import pandas as pd
import json
from nltk.corpus import stopwords
import regex as re

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbook/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd

# Read the CSV file into a DataFrame with explicit encoding
dataset = pd.read_csv("news_summary.csv", encoding='latin1')

# Access the 'text' column of the first row in the 'ctext' column
df = pd.DataFrame()
df['text'] = dataset['ctext']
df['summary'] = dataset['text']

df

,text,summary
0,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...
1,"From her special numbers to TV?appearances, Bo...",Malaika Arora slammed an Instagram user who tr...
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...
...,...,...
4509,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m...",Fruit juice concentrate maker Rasna is eyeing ...
4510,Former cricketer Sachin Tendulkar was spotted ...,Former Indian cricketer Sachin Tendulkar atten...
4511,"Aamir Khan, whose last film Dangal told the st...","Aamir Khan, while talking about reality shows ..."
4512,Maharahstra Power Minister Chandrashekhar Bawa...,The Maharashtra government has initiated an in...


# Data Cleansing using RegEx

In [3]:
# Clean text column
i = 0
for text in df['text']:
    if isinstance(text, str):  # Check if the element is a string
        df['text'][i] = re.sub(r"[!\"#\$%&\(\)\*\+,-\./:;<=>\?@\[\]\^_`{\|}~]", "", text.lower())
    i += 1

# Clean summary column
i = 0
for text in df['summary']:
    if isinstance(text, str):  # Check if the element is a string
        df['summary'][i] = re.sub(r"[!\"#\$%&\(\)\*\+,-\./:;<=>\?@\[\]\^_`{\|}~]", "", text.lower())
    i += 1

In [4]:
stopword = stopwords.words("english")

# Clean stopwords in the text column
i = 0
for text in df['text']:
    if isinstance(text, str):  # Check if the element is a string
        temp = ""
        text = text.split(" ")
        for word in text:
            if word not in stopword:
                temp = temp + " " + word
        df['text'][i] = temp.strip()  # strip to remove leading/trailing spaces
    i += 1

# Clean stopwords in the summary column
i = 0
for text in df['summary']:
    if isinstance(text, str):  # Check if the element is a string
        temp = ""
        text = text.split(" ")
        for word in text:
            if word not in stopword:
                temp = temp + " " + word
        df['summary'][i] = temp.strip()  # strip to remove leading/trailing spaces
    i += 1

In [5]:
df1 = df.dropna(axis=0).reset_index(drop=True)

In [6]:
selected_text = []
selected_sum = []

for i in range(len(df1)):
    if 150 < len(df1['text'][i].split(' ')) < 800:
        if 10 < len(df1['summary'][i].split(' ')) < 50:
            selected_text.append(df1['text'][i])
            selected_sum.append(df1['summary'][i])

In [7]:
df2 = pd.DataFrame()
df2['text'] = selected_text
df2['summary'] = selected_sum

df2.head()

,text,summary
0,daman diu administration wednesday withdrew ci...,administration union territory daman diu revok...
1,special numbers tvappearances bollywood actor ...,malaika arora slammed instagram user trolled d...
2,indira gandhi institute medical sciences igims...,indira gandhi institute medical sciences igims...
3,lashkaretaiba's kashmir commander abu dujana k...,lashkaretaiba's kashmir commander abu dujana k...
4,hotels mumbai indian cities train staff spot s...,hotels maharashtra train staff spot signs sex ...


In [8]:
test_index = int(0.8 * len(df2))

train_data = df2[:test_index]
test_data = df2[test_index:]

print(f"We have {len(train_data)} training samples.")
print(f"We have {len(test_data)} validation samples.")

We have 1976 training samples.
We have 495 validation samples.


In [9]:
# Add START and END tokens to the summaries (y)
special_tokens = ("xstartx", "xendx")

# Handle NaN or float values in "x_sum" column
train_data['summary'] = train_data['summary'].apply(lambda x: special_tokens[0]+' '+str(x)+' '+special_tokens[1] if isinstance(x, str) else x)
test_data['summary'] = test_data['summary'].apply(lambda x: special_tokens[0]+' '+str(x)+' '+special_tokens[1] if isinstance(x, str) else x)

# Accessing a specific element
i = 0
print(train_data['summary'][500])


xstartx central bureau investigation cbi formed special investigation team sit saturday investigate rape murder case minor girl shimla also registered two firs case one girl death one accused police custody xendx


/var/folders/ht/89vfnm2574sb63gbkp_tj13h0000gn/T/ipykernel_10490/3064563492.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['summary'] = train_data['summary'].apply(lambda x: special_tokens[0]+' '+str(x)+' '+special_tokens[1] if isinstance(x, str) else x)
/var/folders/ht/89vfnm2574sb63gbkp_tj13h0000gn/T/ipykernel_10490/3064563492.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['summary'] = test_data['summary'].apply(lambda x: special_tokens[0]+' '+str(x)+' '+special_tokens[1] if

In [10]:
import numpy as np

x_train = np.array(train_data['text'])
y_train = np.array(train_data['summary'])
x_test = np.array(test_data['text'])
y_test = np.array(test_data['summary'])

In [11]:
print(x_test[14])
print('\n' + y_test[14])

delhi chief minister arvind kejriwal sunday denied rift senior leader kumar vishwas aap mla accused latter trying break party bjp's behestaam aadmi party mla amanatullah khan alleged kumar vishwas conspiring break party asked legislators join bjp offer rs 30 crore eachthis prompted kejriwal say attempts made create rift kumar vishwas one founders aap longtime friend kejriwal deputy manish sisodiakhan circulated whatsapp message saying kumar vishwas called aap mlas home monetary offer behest bharatiya janata partykumar vishwas called mlas asked made party convenor said legislator okhla south delhi kumar vishwas gave second offer join bjp ready pay rs 30 crore one themi think done behest bjp khan said adding four aap mlas tasked arranging meetings party legislators kumar vishwasthe four mlas also held meeting unnamed minister khan said citing 10 unnamed mlas revealed kumar vishwas like brother kejriwal tweeted kumar vishwas like younger brothersome people trying show gulf us people enemi

# Tokenizing

In [12]:
## Tokenize text
num_word = 6000
tokenizer = kprocessing.text.Tokenizer(num_words=num_word, lower=True, split=' ', oov_token=None)
tokenizer.fit_on_texts(x_train)
art_vocabulary = {0:"<PAD>"}
art_vocabulary.update(tokenizer.index_word)
print(len(art_vocabulary), "words")

## Create sequence
art_text2seq = tokenizer.texts_to_sequences(x_train)

## Padding sequence
X_train = kprocessing.sequence.pad_sequences(art_text2seq,
                                             maxlen=500, padding='post', truncating="post")

print(X_train.shape[0], "sequences of length", X_train.shape[1])

57896 words
1976 sequences of length 500


In [14]:
## tokenize text
tokenizer.fit_on_texts(x_test)
art_vocabulary.update(tokenizer.index_word)
print(len(art_vocabulary), "words")
## create sequence
art_text2seq = tokenizer.texts_to_sequences(x_test)
## padding sequence
X_test = kprocessing.sequence.pad_sequences(art_text2seq,
                                             maxlen=X_train.shape[1], padding='post', truncating="post")
print(X_test.shape[0], "sequences of length", X_test.shape[1])

66132 words
495 sequences of length 500


In [16]:
# Prepare a tokenizer, again -- by not considering the rare words
x_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=25000) 
x_tokenizer.fit_on_texts(x_train)

# Convert text sequences to integer sequences 
x_train_seq = x_tokenizer.texts_to_sequences(x_train) 
x_test_seq = x_tokenizer.texts_to_sequences(x_test)

# Pad zero upto maximum length
X_train = tf.keras.utils.pad_sequences(x_train_seq,  maxlen=800, padding='post')
X_test = tf.keras.utils.pad_sequences(x_test_seq, maxlen=800, padding='post')

In [17]:
# # Prepare a tokenizer, again -- by not considering the rare words
y_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=8000) 
y_tokenizer.fit_on_texts(y_train)

# Convert text sequences to integer sequences 
y_train_seq = y_tokenizer.texts_to_sequences(y_train) 
y_test_seq = y_tokenizer.texts_to_sequences(y_test) 

# Pad zero upto maximum length
Y_train = tf.keras.utils.pad_sequences(y_train_seq, maxlen=50, padding='post')
Y_test = tf.keras.utils.pad_sequences(y_test_seq, maxlen=50, padding='post')

# Size of vocabulary (+1 for padding token)
# y_voc = y_tokenizer.num_words + 1

# print("Size of vocabulary in Y = {}".format(y_voc))

In [18]:
text_vocab = x_tokenizer.index_word
sum_vocab = y_tokenizer.index_word

In [20]:
tokenizer_json = x_tokenizer.to_json() 

with open('word_corpus.json', 'w', encoding='utf-8') as f:  
      f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [33]:
tokenizer_json = y_tokenizer.to_json() 

with open('label_corpus.json', 'w', encoding='utf-8') as f:  
      f.write(json.dumps(tokenizer_json, ensure_ascii=False))

# Embedding

In [19]:
!pip install gensim
import gensim.downloader
nlp = gensim.downloader.load("glove-wiki-gigaword-300")

In [21]:
## start the matrix (length of vocabulary x vector size) with all 0s
text_embeddings = np.zeros((len(text_vocab)+1, nlp.vector_size), dtype='float32')

for word,idx in text_vocab.items():
  ## update the row with vector
  try:
    text_embeddings[idx] = nlp[word]
  except:
  ## if word not in model then skip and the row stays all 0s
    pass

In [22]:
## start the matrix (length of vocabulary x vector size) with all 0s
sum_embeddings = np.zeros((len(sum_vocab)+1, nlp.vector_size), dtype='float32')

for word,idx in sum_vocab.items():
  ## update the row with vector
  try:
    sum_embeddings[idx] = nlp[word]
  except:
  ## if word not in model then skip and the row stays all 0s
    pass

# Building the Model

In [23]:
text_embeddings.shape[0]-1

57895

In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout

# Encoder
lstm_units = 128  # Reduced the number of LSTM units
embeddings_size = 100  # Reduced the embedding size

encoder_inputs = Input(shape=(X_train.shape[1],))
encoder_embedding = Embedding(input_dim=text_embeddings.shape[0], output_dim=embeddings_size,
                               weights=[text_embeddings[:, :embeddings_size]], trainable=True)(encoder_inputs)
encoder_lstm = LSTM(units=lstm_units, dropout=0.2, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=sum_embeddings.shape[0], output_dim=embeddings_size,
                                weights=[sum_embeddings[:, :embeddings_size]], trainable=True)(decoder_inputs)
decoder_lstm = LSTM(units=lstm_units, dropout=0.2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

# Dense Layer
dense_layer = Dense(units=len(sum_vocab), activation='softmax')
outputs = dense_layer(decoder_outputs)

# Model
model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.01), loss="sparse_categorical_crossentropy", metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 800)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, 800, 100)             5789600   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 100)            1478600   ['input_2[0][0]']             
                                                                                              

In [25]:
training = model.fit(x=[X_train, Y_train[:,:-1]],
                     y=Y_train.reshape(Y_train.shape[0],
                                       Y_train.shape[1],
                                       1)[:,1:],
                     batch_size=64,
                     epochs=50,
                     shuffle=True,
                     verbose=1,
                     validation_split=0.3,

                      )

Epoch 1/50
22/22 [==============================] - 35s 2s/step - loss: 7.3394 - accuracy: 0.3190 - val_loss: 6.4682 - val_accuracy: 0.3285
Epoch 2/50
22/22 [==============================] - 35s 2s/step - loss: 6.0992 - accuracy: 0.3190 - val_loss: 5.9375 - val_accuracy: 0.3294
Epoch 3/50
22/22 [==============================] - 34s 2s/step - loss: 5.6086 - accuracy: 0.3315 - val_loss: 5.5596 - val_accuracy: 0.3552
Epoch 4/50
22/22 [==============================] - 41s 2s/step - loss: 5.3738 - accuracy: 0.3467 - val_loss: 5.5472 - val_accuracy: 0.3567
Epoch 5/50
22/22 [==============================] - 39s 2s/step - loss: 5.2725 - accuracy: 0.3486 - val_loss: 5.5584 - val_accuracy: 0.3566
Epoch 6/50
22/22 [==============================] - 33s 2s/step - loss: 5.1959 - accuracy: 0.3502 - val_loss: 5.5728 - val_accuracy: 0.3569
Epoch 7/50
22/22 [==============================] - 35s 2s/step - loss: 5.1100 - accuracy: 0.3519 - val_loss: 5.5832 - val_accuracy: 0.3548
Epoch 8/50
22/22 [==

In [ ]:
model.save('Model/model_gabungan_171223.h5')

In [26]:
model.evaluate(x=[X_test, Y_test[:,:-1]],
               y=Y_test.reshape(Y_test.shape[0],
                                 Y_test.shape[1],
                                 1)[:,1:])

16/16 [==============================] - 5s 284ms/step - loss: 6.4010 - accuracy: 0.4240


[6.401017665863037, 0.4239950478076935]

## Encoder Model

In [27]:
# Encoder Model
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c], name="Encoder_Model")
encoder_model.summary()

Model: "Encoder_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 800)]             0         
                                                                 
 embedding (Embedding)       (None, 800, 100)          5789600   
                                                                 
 lstm (LSTM)                 [(None, 800, 128),        117248    
                              (None, 128),                       
                              (None, 128)]                       
                                                                 
Total params: 5906848 (22.53 MB)
Trainable params: 5906848 (22.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Decoder Model

In [28]:
# Decoder Model
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding = Embedding(input_dim=sum_embeddings.shape[0], output_dim=embeddings_size,
                              weights=[sum_embeddings[:, :embeddings_size]], trainable=True)(decoder_inputs)

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_outputs = dense_layer(decoder_outputs)

decoder_model = Model(
    inputs=[decoder_inputs] + decoder_states_inputs,
    outputs=[decoder_outputs] + decoder_states,
    name="Decoder_Model"
)
decoder_model.summary()

Model: "Decoder_Model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, None, 100)            1478600   ['input_2[0][0]']             
                                                                                                  
 input_3 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 128)]                0         []                            
                                                                                      

In [29]:
encoder_model.save('Model/encoder_191223.h5')
decoder_model.save('Model/decoder_191223.h5')

/Users/macbook/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Predicting

In [30]:
sample = x_test[1]

clean_sample = re.sub("[-()\"#/@;:<>{}`+=~|.!?,]", "", sample)

temp=""
text=clean_sample.split(" ")
for word in text:
  if word not in stopword:
    temp = temp+" "+word
clean_sample = temp
input_sample = []
input_sample.append(clean_sample.strip())
print(input_sample)

["amazing turn events baahubali 2 conclusion released worldwide april 28 beaten hollywood film circle us box office mind simple featthe circle stars twotime academy awardwinning actor global superstar tom hanks harry potter star heartthrob billions emma watson based novel name circle technothriller one awaited films yearhowever according latest boxoffice figures baahubali 2 given beating tom hanksstarrer us box office baahubali 2 conclusion earned  1013 million rs 65 crore first weekend circle earned  932 million rs 598 crorethis makes baahubali 2 conclusion directed ss rajamouli highest grossing indian film ever american soil us back home baahubali 2 conclusion also breaking records far hindi version film earned title highestgrossing film first weekend india beating sultan dangalstarring prabhas rana daggubati anushka shetty tamannaah sathyaraj ramya krishnan baahubali 2 conclusion fantasy epic sequel 2015 blockbuster baahubali beginning baahubali 2 expected earn whole lot time tell b

In [31]:
with open('word_corpus.json') as f: 
        data = json.load(f) 
        test_tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(data)

input_seq = test_tokenizer.texts_to_sequences(input_sample)

# Padding sequence
x = tf.keras.utils.pad_sequences(input_seq, maxlen=800, padding='post', truncating="post")
# print(tokenizer.index_word)
# print(art_text2seq)
# print(x.shape)
# print(x.shape[0], "sequences of length", x.shape[1])
# print(X_test.shape)

In [32]:
# Predict Manual
# %xmode Verbose
x = x.reshape(1, -1)

# encode X
encoder_out, state_h, state_c = encoder_model.predict(x, verbose=0)

# prepare loop
y_inp = np.array([y_tokenizer.word_index[special_tokens[1]]])
y_expand = np.expand_dims(y_inp, axis=1)
predicted_text = ""
stop = False

while not stop:
    # predict dictionary probability distribution
    outputs = decoder_model.predict([y_expand, state_h, state_c], verbose=0)
    probs, new_state_h, new_state_c = outputs[0], outputs[1], outputs[2]

    # get predicted word
    voc_idx = np.argmax(probs[0, -1, :], axis=0)
    if voc_idx == 0:
        break

    pred_word = y_tokenizer.index_word[voc_idx]

    # check stop
    if (pred_word != special_tokens[1]) and (len(predicted_text.split()) < 50):
        predicted_text = predicted_text + " " + pred_word
    else:
        stop = True

    # next
    y_inp = np.array([voc_idx])
    y_expand = np.expand_dims(y_inp, axis=1)
    state_h, state_c = new_state_h, new_state_c

print(predicted_text)

 india captain england stealing killed flight plane contact next admissions admissions started end end end december reportedly reportedly police police police police


In [ ]:
reverse_target_word_index = y_tokenizer.index_word
reverse_source_word_index = x_tokenizer.index_word
target_word_index = y_tokenizer.word_index

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq, verbose = 0)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['xstartx']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c], verbose = 0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='xendx'):
            decoded_sentence += ' ' + sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'xendx'  or len(decoded_sentence.split()) >= (49)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
decode_sequence(x)